<a href="https://colab.research.google.com/github/pdh21/OCF_PVmapper/blob/main/Example_DeepNet.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

![](https://avatars.githubusercontent.com/u/48357542?s=200&v=4)

## PV mapping Example:
[Peter Hurley](http://pdh21.github.io/) 

In this notebook, we will give a simple example of training a deep net to detect solar panels from remote sensing data. 

As with most neural net projects, we need to:
1. Define training data: input tensors and target tensors.
2. Define a network of layers (or model ) that maps inputs to targets.
3. Configure the learning process by choosing a loss function, an optimizer
some metrics to monitor.
4. Iterate on training data by calling the fit() method of your model.

In [2]:
from google.colab import drive

drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [4]:
cd gdrive/MyDrive/PVmapping/cutouts

/content/gdrive/MyDrive/PVmapping/cutouts


In [5]:
pip install tensorflow-io

In [6]:
import os, sys, math
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
import tensorflow_io as tfio
print("Tensorflow version " + tf.__version__)
AUTO = tf.data.experimental.AUTOTUNE # used in tf.data.Dataset API

Tensorflow version 2.4.1


In [21]:
FILE_PATTERN='./*aerial*.tif'
OUTPUT = '../cutouts_tfrecords/'  # prefix for output file names
TARGET_SIZE = [81,81]


filenames_dataset = tf.data.Dataset.list_files('./*aerial*.tif')

In [11]:
SHARDS = 16


In [12]:
nb_images = len(tf.io.gfile.glob(FILE_PATTERN))
shard_size = math.ceil(1.0 * nb_images / SHARDS)
print("Pattern matches {} images which will be rewritten as {} .tfrec files containing {} images each.".format(nb_images, SHARDS, shard_size))

def decode_tif_and_label(filename):
  bits_aerial = tf.io.read_file(filename)
  bits_binary = tf.io.read_file(tf.strings.regex_replace(filename, "aerial", "binary"))
  image_aerial = tfio.experimental.image.decode_tiff(bits_aerial)[:,:,0:3]
  image_binary = tfio.experimental.image.decode_tiff(bits_binary)[:,:,0:1]


  label = tf.strings.split(tf.expand_dims(filename, axis=-1), sep='_')
  #num = label.values[-2]
  pos_neg=  label.values[0]
  #im_type=label_3.values[1]
  return image_aerial,image_binary,pos_neg

Pattern matches 3999 images which will be rewritten as 16 .tfrec files containing 250 images each.


In [13]:
image_dataset = filenames_dataset.map(decode_tif_and_label)

In [18]:
def recompress_image(image_aerial,image_binary,label):
  height = tf.shape(image_aerial)[0]
  width = tf.shape(image_aerial)[1]
  image_aerial = tf.cast(image_aerial, tf.uint8)
  image_binary = tf.cast(image_binary, tf.uint8)

  image_aerial = tf.image.encode_jpeg(image_aerial, optimize_size=True, chroma_downsampling=False)
  image_binary = tf.image.encode_jpeg(image_binary, optimize_size=True, chroma_downsampling=False)


  return image_aerial,image_binary,label,width,height

dataset3 = image_dataset.map(recompress_image, num_parallel_calls=AUTO)
dataset3 = dataset3.batch(shard_size) # sharding: there will be one "batch" of images per file 

## Write dataset to TFRecord *files*

In [23]:
# Three types of data can be stored in TFRecords: bytestrings, integers and floats
# They are always stored as lists, a single data element will be a list of size 1

def _bytestring_feature(list_of_bytestrings):
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=list_of_bytestrings))

def _int_feature(list_of_ints): # int64
  return tf.train.Feature(int64_list=tf.train.Int64List(value=list_of_ints))

def _float_feature(list_of_floats): # float32
  return tf.train.Feature(float_list=tf.train.FloatList(value=list_of_floats))
  

def to_tfrecord(tfrec_filewriter, img_aerial_bytes, image_binary_bytes, label,height,width):  

  feature = {
      "image_aerial": _bytestring_feature([img_aerial_bytes]), # one image in the list
      "image_binary": _bytestring_feature([image_binary_bytes]),       
      
      # additional (not very useful) fields to demonstrate TFRecord writing/reading of different types of data
      "label":         _bytestring_feature([label]),          # fixed length (1) list of strings, the text label
      "size":          _int_feature([height, width]),         # fixed length (2) list of ints

  }
  return tf.train.Example(features=tf.train.Features(feature=feature))
  
print("Writing TFRecords")
for shard, (image_aerial, image_binary, label, height, width) in enumerate(dataset3):
  # batch size used as shard size here
  shard_size = image_aerial.numpy().shape[0]
  # good practice to have the number of records in the filename
  filename = OUTPUT + "{:02d}-{}.tfrec".format(shard, shard_size)
  
  with tf.io.TFRecordWriter(filename) as out_file:
    for i in range(shard_size):
      example = to_tfrecord(out_file,
                            image_aerial.numpy()[i], # re-compressed image: already a byte string
                            image_binary.numpy()[i], # re-compressed image: already a byte string
                            label.numpy()[i],
                            height.numpy()[i],
                            width.numpy()[i])

      out_file.write(example.SerializeToString())
    print("Wrote file {} containing {} records".format(filename, shard_size))

Writing TFRecords
Wrote file ../cutouts_tfrecords/00-250.tfrec containing 250 records
Wrote file ../cutouts_tfrecords/01-250.tfrec containing 250 records
Wrote file ../cutouts_tfrecords/02-250.tfrec containing 250 records
Wrote file ../cutouts_tfrecords/03-250.tfrec containing 250 records
Wrote file ../cutouts_tfrecords/04-250.tfrec containing 250 records
Wrote file ../cutouts_tfrecords/05-250.tfrec containing 250 records
Wrote file ../cutouts_tfrecords/06-250.tfrec containing 250 records
Wrote file ../cutouts_tfrecords/07-250.tfrec containing 250 records
Wrote file ../cutouts_tfrecords/08-250.tfrec containing 250 records
Wrote file ../cutouts_tfrecords/09-250.tfrec containing 250 records
Wrote file ../cutouts_tfrecords/10-250.tfrec containing 250 records
Wrote file ../cutouts_tfrecords/11-250.tfrec containing 250 records
Wrote file ../cutouts_tfrecords/12-250.tfrec containing 250 records
Wrote file ../cutouts_tfrecords/13-250.tfrec containing 250 records
Wrote file ../cutouts_tfrecord

## Read  from TFRecord Dataset


In [25]:
def read_tfrecord(example):
    features = {
        "image_aerial": tf.io.FixedLenFeature([], tf.string),  # tf.string = bytestring (not text string)
        "image_binary": tf.io.FixedLenFeature([], tf.string),   # shape [] means scalar
        
        # additional (not very useful) fields to demonstrate TFRecord writing/reading of different types of data
        "label":         tf.io.FixedLenFeature([], tf.string),  # one bytestring
        "size":          tf.io.FixedLenFeature([2], tf.int64),  # two integers

    }
    # decode the TFRecord
    example = tf.io.parse_single_example(example, features)
    
    # FixedLenFeature fields are now ready to use: exmple['size']
    # VarLenFeature fields require additional sparse_to_dense decoding
    
    image_aerial = tf.image.decode_jpeg(example['image_aerial'], channels=3)
    image_aerial = tf.reshape(image_aerial, [*TARGET_SIZE, 3])

    image_binary = tf.image.decode_jpeg(example['image_binary'], channels=3)
    image_binary = tf.reshape(image_binary, [*TARGET_SIZE, 3])
    
    
    label  = example['label']
    height = example['size'][0]
    width  = example['size'][1]
    return image_aerial,image_binary, label, height, width
    
# read from TFRecords. For optimal performance, read from multiple
# TFRecord files at once and set the option experimental_deterministic = False
# to allow order-altering optimizations.

option_no_order = tf.data.Options()
option_no_order.experimental_deterministic = False

filenames = tf.io.gfile.glob(OUTPUT + "*.tfrec")
dataset4 = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO)
dataset4 = dataset4.with_options(option_no_order)
dataset4 = dataset4.map(read_tfrecord, num_parallel_calls=AUTO)
dataset4 = dataset4.shuffle(300)